Perform the heatmap generation for all the CSV files provided by *Deltares* and create
* figures showing the data positions and the heatmap (pseudo-color plot)
* netCDF files (one per species) storing the heatmap.

`doplot` can be turned to 'false' to avoid creating the figures.

In [1]:
using DIVAnd
using PyPlot
using Proj4
using DelimitedFiles
using PyCall
using Dates
using NCDatasets
include("../scripts/PhytoInterp.jl")
include("../scripts/PhytoInterpPlot.jl")
doplot = true

true

## Packages for plotting

In [2]:
if doplot
    ccrs = pyimport("cartopy.crs")
    gridliner = pyimport("cartopy.mpl.gridliner")
    cfeature = pyimport("cartopy.feature")
    mticker = pyimport("matplotlib.ticker")
    myproj = ccrs.PlateCarree()
    coast = cfeature.GSHHSFeature(scale="full");
    mpl = pyimport("matplotlib");
    cartopyticker = pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
end 

PyObject <cartopy.mpl.ticker.LatitudeFormatter object at 0x7f7a2286af90>

## Files and directories

In [3]:
csvdir = "/data/EMODnet/Biology/phytoplankton/csv/"
datadir = "./data/"
datafilelist = readdir(csvdir);
@info("Working on $(length(datafilelist)) files");
figdir = "./figures/"
outputdir = "../product/"
speciesname = "Biddulphia_sinensis"
isdir(datadir) ? " " : mkpath(datadir)
isdir(figdir) ? " " : mkpath(figdir);

┌ Info: Working on 200 files
└ @ Main In[3]:4


## Prepare mask
### Interpolation grid

In [4]:
longrid = -2.:0.1:10.
latgrid = 51.:0.1:56.

51.0:0.1:56.0

### Download file

In [5]:
bathname = joinpath(datadir, "gebco_30sec_4.nc")
if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/RSwm4HPHImdZoQP/download", bathname)
else
    @info("Bathymetry file already downloaded")
end

┌ Info: Bathymetry file already downloaded
└ @ Main In[5]:5


### Read bathymetry

In [6]:
bx, by, b = load_bath(bathname, true, longrid, latgrid)
@show size(b)

if doplot
    plot_bathymetry(bx, by, b, joinpath(figdir, "northsea_bathy"))
end

size(b) = (121, 51)


### Metrics

In [7]:
mask, (pm, pn),(xi, yi) = DIVAnd.DIVAnd_rectdom(longrid, latgrid);
xi, yi, mask = DIVAnd.load_mask(bathname, true, longrid, latgrid, 0.0);
xx, yy = ndgrid(xi, yi);

In [8]:
if doplot
    plot_mask(bx, by, mask, joinpath(figdir, "northsea_mask"))
end

## Main loop

In [9]:
datafilelist = ["/data/EMODnet/Biology/phytoplankton/csv/Biddulphia sinensis-1995-2020.csv"]
figdir

"./figures/"

In [24]:
include("../scripts/PhytoInterpPlot.jl")

plot_error (generic function with 3 methods)

In [25]:
for datafile in datafilelist
    speciesname = get_species_name(basename(datafile))
    speciesslug = get_species_slug(basename(datafile))
    @info("Working on $(speciesname)")
    @info(speciesslug)
        
    # Data reading
    dates, lons, lats, occurs = read_data_phyto(joinpath(csvdir, datafile));
    
    # Conversion of coordinates
    # We have to go from `EPSG:32361` to `WGS84`.         
    lon, lat = transform_coords(lons, lats)
    
    # Plot presence/absence
    if doplot
        @info("Plotting")
        make_plot_presence_absence(lon, lat, occurs, 
            joinpath(figdir, "$(speciesslug)_data_presence_absence.png"))
    end
    
    data_presence = occurs .== 1;
    data_absence = .!(data_presence);
    
    @info("Presence data: $(sum(data_presence)), absence data: $(sum(data_absence))")
    
    # Compute heatmap
    inflation = ones(length(lon));
    @time dens1, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon, lat), inflation, 0.1);
    @time dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_presence], lat[data_presence]), inflation[data_presence], 0.1);
    @time dens3, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_absence], lat[data_absence]), inflation[data_absence], 0.1);
    
    if doplot
        
        plot_heatmap(longrid, latgrid, dens1, lon, lat, occurs,
            "$(speciesname): heatmap with 'presence' data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_presence.png"), 0., 1.)
        
        plot_heatmap(longrid, latgrid, dens2, lon, lat, occurs,
            "$(speciesname): heatmap with 'absence' data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_absence.png"), 0., 1.)
        
        plot_heatmap(longrid, latgrid, dens3, lon, lat, occurs,
            "$(speciesname): heatmap with all data points", 
            joinpath(figdir, "$(speciesslug)_heatmap_all.png"), 0., 1.)
        
        plot_heatmap(longrid, latgrid, dens2 ./ (dens2.+dens3), lon, lat, occurs,
            "$(speciesname): normalised heatmap (presence/presence+absence)", 
            joinpath(figdir, "$(speciesslug)_heatmap_relative2.png"), 0., 1.)
    end
    
    # Write in netCDF
    create_nc_results(joinpath(outputdir, "$(speciesslug)_heatmap.nc"), 
        longrid, latgrid, dens2 ./ (dens2 .+ dens3), speciesname);
 
end

┌ Info: Working on Biddulphia sinensis
└ @ Main In[25]:4
┌ Info: Biddulphia_sinensis
└ @ Main In[25]:5
┌ Info: Plotting
└ @ Main In[25]:16
┌ Info: Presence data: 6911, absence data: 16818
└ @ Main In[25]:24
┌ Warning: Data array size (23729,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/ZfEqh/src/DIVAnd_heatmap.jl:48


  1.703600 seconds (865.35 k allocations: 1.158 GiB, 3.96% gc time)
  6.769677 seconds (617.27 k allocations: 4.722 GiB, 2.66% gc time)


┌ Warning: Data array size (16818,) is larger then maximum 10000. Superobservations will be created. To avoid, increase nmax to the desired number of superobs
└ @ DIVAnd /home/ctroupin/.julia/packages/DIVAnd/ZfEqh/src/DIVAnd_heatmap.jl:48


  1.524629 seconds (636.48 k allocations: 1018.994 MiB, 2.90% gc time)
